In [1]:
import numpy as np
import scipy.linalg as sla
import scipy.sparse as sps
import matplotlib.pyplot as plt

# Matrix inverse square root

In [7]:
f = lambda x: 1.0 / np.sqrt(x)

N = 500

U, ss, _ = np.linalg.svd(np.random.randn(N,N))
A = U @ np.diag(ss) @ U.T
f_of_A = U @ np.diag(f(ss)) @ U.T

print('np.linalg.cond(A)=', np.linalg.cond(A))
print('cond(f_of_A)=', np.linalg.cond(f_of_A))

np.linalg.cond(A)= 1649.9611486593678
cond(f_of_A)= 40.619713793422726


In [3]:
apply_A = lambda x: A @ x
b = np.random.randn(N)
x_true = f_of_A @ b

qq = [np.zeros(b.shape), b / np.linalg.norm(b)]
alphas = []
betas = [0.0]

for ii in range(50):
    Q = np.asarray(qq[1:]).T.reshape((N, -1))
    
    q_prev = qq[-2]
    q_cur = qq[-1]
    v = apply_A(q_cur)
    alpha = q_cur.T @ v
    v = v - betas[-1] * q_prev - alpha * q_cur
    beta = np.linalg.norm(v)
    q_next = v / beta
    qq.append(q_next)
    alphas.append(alpha)
    betas.append(beta)
    
    evals_T, evecs_T = sla.eigh_tridiagonal(alphas, betas[1:-1])
    f_of_T = evecs_T @ np.diag(f(evals_T)) @ evecs_T.T
    x = Q @ (f_of_T @ (Q.T @ b))
    relres = np.linalg.norm(x - f_of_A @ b) / np.linalg.norm(b)
    relerr = np.linalg.norm(x_true - x) / np.linalg.norm(x_true)
    print('ii=', ii, ', relres=', relres, ', relerr=', relerr)
    
T = sps.diags([betas[1:-1], alphas, betas[1:-1]], [-1, 0, 1])

ii= 0 , relres= 0.530334705733151 , relerr= 0.8641267673976699
ii= 1 , relres= 0.504130438725241 , relerr= 0.8214295644864118
ii= 2 , relres= 0.4863039799305274 , relerr= 0.7923831527658584
ii= 3 , relres= 0.47139764968722864 , relerr= 0.7680947951915661
ii= 4 , relres= 0.45922185792941167 , relerr= 0.7482555739253576
ii= 5 , relres= 0.44848323888432584 , relerr= 0.7307580802455563
ii= 6 , relres= 0.44045333521385294 , relerr= 0.7176741643217663
ii= 7 , relres= 0.43349144675874246 , relerr= 0.7063304711772999
ii= 8 , relres= 0.4268003498802266 , relerr= 0.6954280055202892
ii= 9 , relres= 0.4206558523412527 , relerr= 0.685416168206538
ii= 10 , relres= 0.4158977582364978 , relerr= 0.677663335074428
ii= 11 , relres= 0.41195366399349026 , relerr= 0.6712368323928615
ii= 12 , relres= 0.40867190929966024 , relerr= 0.6658895450207272
ii= 13 , relres= 0.40578515281188016 , relerr= 0.6611858672770631
ii= 14 , relres= 0.4031613809098829 , relerr= 0.6569106963188898
ii= 15 , relres= 0.400614607890

# Matrix logarithm

In [20]:
f = lambda x: np.log(x)
inv_f = lambda x: np.exp(x)

N = 500

U, ss, _ = np.linalg.svd(np.random.randn(N,N))
A = U @ np.diag(ss) @ U.T
f_of_A = U @ np.diag(f(ss)) @ U.T

print('np.linalg.cond(A)=', np.linalg.cond(A))
print('cond(f_of_A)=', np.linalg.cond(f_of_A))

np.linalg.cond(A)= 2291.3623540118197
cond(f_of_A)= 137.02428856788455


In [39]:
apply_A = lambda x: A @ x
b = np.random.randn(N)
x_true = f_of_A @ b

qq = [np.zeros(b.shape), b / np.linalg.norm(b)]
alphas = []
betas = [0.0]
x_superold = np.zeros(N)
x_old = np.zeros(N)
x = np.zeros(N)
for ii in range(50):
    x_superold = x_old
    x_old = x
    Q = np.asarray(qq[1:]).T.reshape((N, -1))
    
    q_prev = qq[-2]
    q_cur = qq[-1]
    v = apply_A(q_cur)
    alpha = q_cur.T @ v
    v = v - betas[-1] * q_prev - alpha * q_cur
    beta = np.linalg.norm(v)
    q_next = v / beta
    qq.append(q_next)
    alphas.append(alpha)
    betas.append(beta)
    
    evals_T, evecs_T = sla.eigh_tridiagonal(alphas, betas[1:-1])
    f_of_T = evecs_T @ np.diag(f(evals_T)) @ evecs_T.T
    inv_f_of_T = evecs_T @ np.diag(inv_f(evals_T)) @ evecs_T.T
    x = Q @ (f_of_T @ (Q.T @ b))
    b2 = Q @ (inv_f_of_T @ (Q.T @ x))
    T = sps.diags([betas[1:-1], alphas, betas[1:-1]], [-1, 0, 1])
#     err_est = np.linalg.norm(A @ Q - Q @ T) / np.linalg.norm(A @ Q)
#     err_est = np.linalg.norm(b2 - b) / np.linalg.norm(b)
#     err_est = np.linalg.norm(x - x_old) / np.linalg.norm(x - x_superold)
    relres = np.linalg.norm(x - f_of_A @ b) / np.linalg.norm(b)
    relerr = np.linalg.norm(x_true - x) / np.linalg.norm(x_true)
    print('ii=', ii, ', relres=', relres, ', relerr=', relerr)#, ', err_est=', err_est)
    
T = sps.diags([betas[1:-1], alphas, betas[1:-1]], [-1, 0, 1])

ii= 0 , relres= 1.2505303140937858 , relerr= 0.4621069398239284
ii= 1 , relres= 0.6863550567476125 , relerr= 0.25362794594560406
ii= 2 , relres= 0.46006565830856233 , relerr= 0.1700075008842261
ii= 3 , relres= 0.3421623945070966 , relerr= 0.12643885179471465
ii= 4 , relres= 0.2767799715888753 , relerr= 0.10227816489852544
ii= 5 , relres= 0.23773130730985045 , relerr= 0.08784855967358657
ii= 6 , relres= 0.21276308560015392 , relerr= 0.0786220831963055
ii= 7 , relres= 0.1933075562481418 , relerr= 0.07143270519387933
ii= 8 , relres= 0.1786101838196282 , relerr= 0.06600160310874936
ii= 9 , relres= 0.16671654217094736 , relerr= 0.06160656023478498
ii= 10 , relres= 0.15733556585556033 , relerr= 0.05814001951297475
ii= 11 , relres= 0.14833891806882551 , relerr= 0.05481549924301674
ii= 12 , relres= 0.1396393293503689 , relerr= 0.05160075084779211
ii= 13 , relres= 0.1300029940205166 , relerr= 0.04803984762121004
ii= 14 , relres= 0.12038436830825734 , relerr= 0.044485488607990196
ii= 15 , relres

In [36]:
np.linalg.norm(A @ Q - Q @ T) / np.linalg.norm(A @ Q)

0.0546488991728871